# Load the Dataset

In [71]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/breast-cancer-wisconsin-data")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/uciml/breast-cancer-wisconsin-data/versions/2


In [72]:
df =  pd.read_csv("/content/drive/MyDrive/PYTORCH/data.csv")

In [73]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [74]:
df.shape

(569, 33)

In [75]:
df.drop(columns= ["id" ,"Unnamed: 32"] , inplace = True)

In [76]:
df.shape


(569, 31)

# Train Test Split

In [77]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('diagnosis', axis=1), df['diagnosis'], test_size=0.2, random_state=42)

# Scaling

In [78]:
scaler =  StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [79]:
X_train

array([[-1.44075296, -0.43531947, -1.36208497, ...,  0.9320124 ,
         2.09724217,  1.88645014],
       [ 1.97409619,  1.73302577,  2.09167167, ...,  2.6989469 ,
         1.89116053,  2.49783848],
       [-1.39998202, -1.24962228, -1.34520926, ..., -0.97023893,
         0.59760192,  0.0578942 ],
       ...,
       [ 0.04880192, -0.55500086, -0.06512547, ..., -1.23903365,
        -0.70863864, -1.27145475],
       [-0.03896885,  0.10207345, -0.03137406, ...,  1.05001236,
         0.43432185,  1.21336207],
       [-0.54860557,  0.31327591, -0.60350155, ..., -0.61102866,
        -0.3345212 , -0.84628745]])

# Label Encoding Target

In [80]:
encoder  =  LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# Numpy array to tensors


In [81]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [82]:
X_train_tensor.shape

torch.Size([455, 30])

# Defining Model

In [83]:
class MySimpleNN():

  def __init__(self , X):
    self.weights =  torch.rand(X.shape[1] , 1 , dtype =  torch.float64 , requires_grad=True)
    self.bias = torch.rand(1, dtype=torch.float64, requires_grad=True)

  def forward(self , X):
    z = torch.matmul(X , self.weights) + self.bias
    y_pred =  torch.sigmoid(z)
    return y_pred

  def loss(self , y_pred , y_true):

    epsilon =  1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1.0 - epsilon)
    loss = - (y_true * torch.log(y_pred + epsilon) + (1 - y_true) * torch.log(1 - y_pred + epsilon))
    return loss.mean()


# Important Params

In [84]:
learning_rate =  0.1
epochs = 100

# Training Pipeline

In [85]:
model =  MySimpleNN(X_train_tensor)


for epoch in range(epochs):
  # Foward pass
  y_pred = model.forward(X_train_tensor)

  # Loss calculate
  loss = model.loss(y_pred , y_train_tensor)
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

  # backwqard pass
  loss.backward()

  # update weights
  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  # zero gradient
  model.weights.grad.zero_()
  model.bias.grad.zero_()


Epoch: 1, Loss: 3.718755632039063
Epoch: 2, Loss: 3.6021847440962307
Epoch: 3, Loss: 3.4817508078208967
Epoch: 4, Loss: 3.3580766197065075
Epoch: 5, Loss: 3.229598301066717
Epoch: 6, Loss: 3.096253826282265
Epoch: 7, Loss: 2.9580483061412335
Epoch: 8, Loss: 2.813679946975939
Epoch: 9, Loss: 2.6646776346309324
Epoch: 10, Loss: 2.5098365955527293
Epoch: 11, Loss: 2.3489860607898794
Epoch: 12, Loss: 2.1849817177227453
Epoch: 13, Loss: 2.0203917919929695
Epoch: 14, Loss: 1.8602068823685076
Epoch: 15, Loss: 1.7031121265806695
Epoch: 16, Loss: 1.548417629696713
Epoch: 17, Loss: 1.4023279538812072
Epoch: 18, Loss: 1.2699345836099925
Epoch: 19, Loss: 1.1540312331364206
Epoch: 20, Loss: 1.0563769067764526
Epoch: 21, Loss: 0.9775353724507891
Epoch: 22, Loss: 0.9166713509575389
Epoch: 23, Loss: 0.8716058270933554
Epoch: 24, Loss: 0.8392639062657477
Epoch: 25, Loss: 0.8163555810576374
Epoch: 26, Loss: 0.7999602958323018
Epoch: 27, Loss: 0.7878257499319204
Epoch: 28, Loss: 0.7783986145467059
Epoch:

In [86]:
model.weights

tensor([[ 0.4052],
        [-0.1212],
        [-0.3964],
        [-0.2187],
        [ 0.2320],
        [-0.3827],
        [ 0.1196],
        [ 0.0674],
        [-0.0792],
        [ 0.3347],
        [-0.4469],
        [-0.0576],
        [ 0.4573],
        [-0.2587],
        [-0.1023],
        [-0.0267],
        [ 0.3301],
        [ 0.2033],
        [-0.0067],
        [-0.1524],
        [ 0.2965],
        [ 0.2023],
        [ 0.4080],
        [ 0.1886],
        [ 0.0291],
        [-0.0422],
        [-0.1685],
        [-0.6037],
        [ 0.1242],
        [-0.0296]], dtype=torch.float64, requires_grad=True)

In [87]:
model.bias

tensor([-0.4111], dtype=torch.float64, requires_grad=True)

In [94]:
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.7).float()
  y_pred

In [95]:
Accuracy =  (y_pred.round() == y_test_tensor).float().mean()
Accuracy.item()

0.6206524968147278